# Dashboarding 2019-nCoV Development with Python Scraping and Pyecharts
----


## 0. Preface
* Tencent weiste changed it's JSON coding since the beginning of Feb 2020.
* Dashboard with Baidu's pyecharts library - way easier than ever.
* Data souce: [Tencent Live Tracking of 2019-nCoV](https://news.qq.com/zt2020/page/feiyan.htm)


## Part 1. Data Entry

### 1.1 Load Libraries

In [1]:
import time 
import json
import requests
from datetime import datetime
import pandas as pd 
import numpy as np 

### 1.2 Catch the Data Stream

In [2]:
def catch_data():
    url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5'
    reponse = requests.get(url=url).json()
    # return the Data Dictionary
    data = json.loads(reponse['data'])
    return data


In [3]:
data = catch_data()
data.keys()

dict_keys(['chinaTotal', 'chinaAdd', 'lastUpdateTime', 'areaTree', 'chinaDayList', 'chinaDayAddList', 'isShowAdd'])

### 1.3 Dataset Peek

In [4]:
# Dataset includes ["chinaTotal","chinaAdded","lastUpdateTime","children","dailyData","dailyAdded"]

lastUpdateTime = data['lastUpdateTime']
chinaTotal = data['chinaTotal']
chinaAdd = data['chinaAdd']
print('Last Update Time:', lastUpdateTime)
print('Current China Total:', chinaTotal)
print('Current China Added:', chinaAdd)


Last Update Time: 2020-02-08 18:22:12
Current China Total: {'confirm': 34662, 'suspect': 27657, 'dead': 724, 'heal': 2220}
Current China Added: {'confirm': 3451, 'suspect': 1298, 'dead': 87, 'heal': 678}


### 1.4 China Dataset Processing - Step One

In [5]:
# Complex Dataset structure! So print 'em out to pay a look.
areaTree = data['areaTree']
# China Dataset
china_data = areaTree[0]['children']
china_list = []
for a in range(len(china_data)):
    province = china_data[a]['name']
    province_list = china_data[a]['children']
    for b in range(len(province_list)):
        city = province_list[b]['name']
        total = province_list[b]['total']
        today = province_list[b]['today']
        china_dict = {}
        china_dict['province'] = province
        china_dict['city'] = city
        china_dict['total'] = total
        china_dict['today'] = today
        china_list.append(china_dict)
        
china_data = pd.DataFrame(china_list)
china_data.head()


,province,city,total,today
0,湖北,武汉,"{'confirm': 13603, 'suspect': 0, 'dead': 545, ...","{'confirm': 1985, 'suspect': 0, 'dead': 0, 'he..."
1,湖北,孝感,"{'confirm': 2313, 'suspect': 0, 'dead': 26, 'h...","{'confirm': 172, 'suspect': 0, 'dead': 0, 'hea..."
2,湖北,黄冈,"{'confirm': 2041, 'suspect': 0, 'dead': 36, 'h...","{'confirm': 144, 'suspect': 0, 'dead': 0, 'hea..."
3,湖北,随州,"{'confirm': 953, 'suspect': 0, 'dead': 9, 'hea...","{'confirm': 38, 'suspect': 0, 'dead': 0, 'heal..."
4,湖北,荆州,"{'confirm': 941, 'suspect': 0, 'dead': 11, 'he...","{'confirm': 56, 'suspect': 0, 'dead': 0, 'heal..."


### 1.5 China Dataset Processing - Step Two

In [6]:
# Define a few data-processing functions

def confirm(x):
    confirm = eval(str(x))['confirm']
    return confirm
def suspect(x):
    suspect = eval(str(x))['suspect']
    return suspect
def dead(x):
    dead = eval(str(x))['dead']
    return dead
def heal(x):
    heal =  eval(str(x))['heal']
    return heal

# Functions mapping
china_data['confirm'] = china_data['total'].map(confirm)
china_data['suspect'] = china_data['total'].map(suspect)
china_data['dead'] = china_data['total'].map(dead)
china_data['heal'] = china_data['total'].map(heal)
china_data['addconfirm'] = china_data['today'].map(confirm)
china_data['addsuspect'] = china_data['today'].map(suspect)
china_data['adddead'] = china_data['today'].map(dead)
china_data['addheal'] = china_data['today'].map(heal)
china_data = china_data[["province","city","confirm","suspect","dead","heal","addconfirm","addsuspect","adddead","addheal"]]
china_data.head()


,province,city,confirm,suspect,dead,heal,addconfirm,addsuspect,adddead,addheal
0,湖北,武汉,13603,0,545,698,1985,0,0,0
1,湖北,孝感,2313,0,26,38,172,0,0,0
2,湖北,黄冈,2041,0,36,105,144,0,0,0
3,湖北,随州,953,0,9,23,38,0,0,0
4,湖北,荆州,941,0,11,29,56,0,0,0


### 1.6 International Dataset Processing with Country.xlsx spreadsheet

In [7]:
global_data = pd.DataFrame(data['areaTree'])
global_data['confirm'] = global_data['total'].map(confirm)
global_data['suspect'] = global_data['total'].map(suspect)
global_data['dead'] = global_data['total'].map(dead)
global_data['heal'] = global_data['total'].map(heal)
global_data['addconfirm'] = global_data['today'].map(confirm)
global_data['addsuspect'] = global_data['today'].map(suspect)
global_data['adddead'] = global_data['today'].map(dead)
global_data['addheal'] = global_data['today'].map(heal)
world_name = pd.read_excel("./res/country.xlsx")
global_data = pd.merge(global_data,world_name,left_on ="name",right_on = "name_cn",how="inner")
global_data = global_data[["name","name_en","confirm","suspect","dead","heal","addconfirm","addsuspect","adddead","addheal"]]
global_data.head()


,name,name_en,confirm,suspect,dead,heal,addconfirm,addsuspect,adddead,addheal
0,中国,China,34662,27657,724,2220,3451,0,0,0
1,韩国,"Korea,Republic of",24,0,0,2,0,0,0,0
2,澳大利亚,Australia,15,0,0,5,0,0,0,0
3,德国,Germany,13,0,0,0,0,0,0,0
4,法国,France,11,0,0,0,0,0,0,0


### 1.7 Daily Dataset Processing

In [8]:
chinaDayList = pd.DataFrame(data['chinaDayList'])
chinaDayList = chinaDayList[['date','confirm','suspect','dead','heal']]
chinaDayList.head()


,date,confirm,suspect,dead,heal
0,01.13,41,0,1,0
1,01.14,41,0,1,0
2,01.15,41,0,2,5
3,01.16,45,0,2,8
4,01.17,62,0,2,12


### 1.8 Daily Addition Dataset Processing

In [9]:
chinaDayAddList = pd.DataFrame(data['chinaDayAddList'])
chinaDayAddList = chinaDayAddList[['date','confirm','suspect','dead','heal']]
chinaDayAddList.head()


,date,confirm,suspect,dead,heal
0,01.20,77,27,0,0
1,01.21,149,53,3,0
2,01.22,131,257,8,0
3,01.23,259,680,8,6
4,01.24,444,1118,16,3


## Part 2. Visualization

### 2.1 China Total Detailed with Pie Charts

In [10]:
# Import all charts of Baidu's Pyecharts library
from pyecharts.charts import * 
from pyecharts import options as opts

# Import Themes of Pyecharts if needed (can ignore this line if not used)
from pyecharts.globals import ThemeType


In [11]:
# Pie chart for China Total
total_pie = Pie(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS,width = '550px',height ='350px'))  # Set theme, canvas size
total_pie.add("",[list(z) for z in zip(chinaTotal.keys(), chinaTotal.values())],
            center=["50%", "50%"], # Location of the pie chart
            radius=[50, 80])       # ID and OD of the pie chart
total_pie.set_global_opts(
            title_opts=opts.TitleOpts(title="China Total",subtitle=("Till "+lastUpdateTime)))
total_pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{c}"))  # Label format
total_pie.render_notebook()


In [12]:
# Another Pie Chart for Newly Added
totaladd_pie = Pie(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS,width = '550px',height ='350px'))  # Set theme, canvas size
totaladd_pie.add("",[list(z) for z in zip(chinaAdd.keys(), chinaAdd.values())],
            center=["50%", "50%"],
            radius=[50, 80])
totaladd_pie.set_global_opts(
            title_opts=opts.TitleOpts(title="China Added"))
totaladd_pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{c}"))  # Label format
totaladd_pie.render_notebook()


## 2.2 Global Heat Maps

In [13]:
# World map
world_map = Map(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS))
world_map.add("",[list(z) for z in zip(list(global_data["name_en"]), list(global_data["confirm"]))], "world",is_map_symbol_show=False)
world_map.set_global_opts(title_opts=opts.TitleOpts(title="2019_nCoV Global Heat Map"),
                          visualmap_opts=opts.VisualMapOpts(is_piecewise=True,
                          pieces = [
                        {"min": 101 , "label": '>100'}, # Missing max means unlimited max
                        {"min": 10, "max": 100, "label": '10-100'},
                        {"min": 0, "max": 9, "label": '0-9' }]))
world_map.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
world_map.render_notebook()

## 2.3 China Heat Map

In [14]:
# Data processing
area_data = china_data.groupby("province")["confirm"].sum().reset_index()
area_data.columns = ["province","confirm"]


In [15]:
area_map = Map(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS))
area_map.add("",[list(z) for z in zip(list(area_data["province"]), list(area_data["confirm"]))], "china",is_map_symbol_show=False)
area_map.set_global_opts(title_opts=opts.TitleOpts(title="2019-nCoV China Heat Map"),visualmap_opts=opts.VisualMapOpts(is_piecewise=True,
                pieces = [
                        {"min": 1001 , "label": '>1000',"color": "#893448"}, # Missing max means unlimited max
                        {"min": 500, "max": 1000, "label": '500-1000',"color": "#ff585e"},
                        {"min": 101, "max": 499, "label": '101-499',"color": "#fb8146"},
                        {"min": 10, "max": 100, "label": '10-100',"color": "#ffb248"},
                        {"min": 0, "max": 9, "label": '0-9',"color" : "#fff2d1" }]))
area_map.render_notebook()


### 2.4 Plot Daily Trend
### 2.4.1 Daily CDF of Healed/Dead Cases in China

In [16]:
line1 = Line(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS))
line1.add_xaxis(list(chinaDayList["date"]))
line1.add_yaxis("Healed",list(chinaDayList["heal"]),is_smooth=True)
line1.add_yaxis("Dead", list(chinaDayList["dead"]),is_smooth=True)
line1.set_global_opts(title_opts=opts.TitleOpts(title="Line1-Healed vs Death Toll"))
line1.render_notebook()


### 2.4.1 Daily CDF for Confirmed/Suspected Cases in China

In [17]:
line2 = Line(init_opts=opts.InitOpts(theme=ThemeType.SHINE))
line2.add_xaxis(list(chinaDayList["date"]))
line2.add_yaxis("Confirmed",list(chinaDayList["confirm"]))
line2.add_yaxis("Suspected", list(chinaDayList["suspect"]))
line2.set_global_opts(title_opts=opts.TitleOpts(title="Line2-Confirmed vs Suspected"))
line2.render_notebook()


### 2.5 Daily Added in China

In [18]:
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS,width = '900px',height ='400px'))
bar.add_xaxis(list(chinaDayAddList["date"]))
bar.add_yaxis("Confirmed", list(chinaDayAddList["confirm"]))
bar.add_yaxis("Suspected", list(chinaDayAddList["suspect"]))
bar.add_yaxis("Dead", list(chinaDayAddList["dead"]))
bar.add_yaxis("Healed", list(chinaDayAddList["heal"]))
bar.set_global_opts(title_opts=opts.TitleOpts(title="Trend of Daily Added Cases"))
bar.render_notebook()


## Part 3. All-in-one Charts into Webpage

In [19]:
page = Page()
page.add(total_pie)
page.add(totaladd_pie)
page.add(world_map)
page.add(area_map)
page.add(line1)
page.add(line2)
page.add(bar)
page.render("2019_nCoV_Visualization_Totalized.html")


'C:\\Users\\Marcus\\Documents\\GitHub\\2019nCoV\\2019_nCoV_Visualization_Totalized.html'

----
## Courtesy of Hakuna_Matata_001

————————————————

* Copyrights Reserved by the CSDN Blogger - Hakuna_Matata_001.
* Abided by Copyright protocols of CC 4.0 BY-SA. 
* Please cite the article origin when quoting.

  Code of Origin: https://blog.csdn.net/weixin_43130164/article/details/104113559